## Выполнение тестового задания для g-plans.com
#### ovesgur@gmail.com

<b>Задание(кратко):</b> <br>
    Из входных csv файлов мониторинга источника трафика необходимо подготовить набор для создания отчетов. <br>
    Файлы - ad_sys.csv, billing_sys.csv, internal_events.csv

<b>Table of contents</b> </br>
1. [Импорт](#import)
2. [Создание класса подготовки таблицы](#creation_class)
3. [Создание объекта класса](#creation_obj)
4. [Размышления и результаты](#results)

<a id="import"></a>
### Import

In [1]:
import pyodbc

### Создание класса подготовки таблицы
<a id="creation_class"></a>

In [2]:
class view_creator:
    def __init__(self, files, sql_server_nm, db_nm, db_tables_nm):
        conn = self.connect_db(sql_server_nm, db_nm)
        self.create_tables(conn, db_tables_nm)
        self.insert_data_bulk(conn, files, db_tables_nm)
        self.union_tables(conn, db_tables_nm)
        self.drop_nulls(conn, db_tables_nm)
        conn.close
        
    def connect_db(self, sql_server_nm, db_nm):
        conn = pyodbc.connect("DRIVER={ODBC Driver 17 for SQL Server};server=" + sql_server_nm + "; database=revenue_spent_db; trusted_connection=yes;",autocommit=True)
        return conn
    
    def create_tables(self, conn, db_tables_nm):
        
        cursor = conn.cursor()
        cursor.execute('''
        CREATE TABLE ''' + db_tables_nm[0] + ''' (
        date DATE,
        revenue REAL,
        spent REAL,
        _collected_at VARCHAR(100),
        affid INT); ''')
        cursor.close
        
        cursor = conn.cursor()
        cursor.execute('''
        CREATE TABLE ''' + db_tables_nm[1] + ''' (
        created_at VARCHAR(100),
        affid REAL,
        amount REAL,
        uid VARCHAR(100)
        ); ''' )
        cursor.close
                
        cursor = conn.cursor()
        cursor.execute('''
        CREATE TABLE ''' + db_tables_nm[2] + ''' (
        created_at VARCHAR(100),
        affid REAL,
        revenue REAL,
        user_id VARCHAR(100)
        ); ''' )
        
        cursor = conn.cursor()
        cursor.execute('''
        CREATE TABLE ''' + db_tables_nm[3] + ''' (
        system VARCHAR(1000),
        affid REAL,
        revenue REAL,
        spent REAL,
        date DATETIME
        ); ''' )
        cursor.close
                    
        conn.commit()
        
    
    def insert_data_bulk(self, conn, files, db_tables_nm):
        qry = "BULK INSERT " + db_tables_nm[0] + " FROM '" + files[0] + "' WITH (FIRSTROW = 2, FIELDTERMINATOR =',', ROWTERMINATOR = '0x0a')"
        
        cursor = conn.cursor()
        success = cursor.execute(qry)
                       
        conn.commit()
        cursor.close
                       
        qry = "BULK INSERT " + db_tables_nm[1] + " FROM '" + files[1] + "' WITH (FIRSTROW = 2, FIELDTERMINATOR =',', ROWTERMINATOR = '0x0a')"
        
        cursor = conn.cursor()
        success = cursor.execute(qry)
                       
        conn.commit()
        cursor.close
                       
        qry = "BULK INSERT " + db_tables_nm[2] + " FROM '" + files[2] + "' WITH (FIRSTROW = 2, FIELDTERMINATOR =',', ROWTERMINATOR = '0x0a')"
        
        cursor = conn.cursor()
        success = cursor.execute(qry)
                       
        conn.commit()
        cursor.close
    
    def union_tables(self, conn, db_tables_nm):
        cursor = conn.cursor()
        cursor.execute('''
        INSERT INTO ''' + db_tables_nm[3] + ''' 
        SELECT 1 AS id_system, affid, revenue, spent, CONVERT(DATETIME, REPLACE(LEFT(adv._collected_at, CHARINDEX('.',adv._collected_at)-1),'_',' '), 102 ) date
        FROM dbo.ad_sys adv
        ''')
        cursor.close
        
        cursor = conn.cursor()
        cursor.execute('''
        INSERT INTO ''' + db_tables_nm[3] + ''' 
        SELECT 2 AS id_system, affid, amount as revenue, 0 as spent, CONVERT(DATETIME, REPLACE(LEFT(bil_sys.created_at, CHARINDEX('U',bil_sys.created_at)-1),'_',' '), 102 )
        FROM dbo.billing_sys bil_sys
        ''')
        cursor.close
        
        cursor = conn.cursor()
        cursor.execute('''
        INSERT INTO ''' + db_tables_nm[3] + ''' 
        SELECT 3 AS id_system, affid, revenue, 0 as spent, CONVERT(DATETIME, REPLACE(LEFT(ie_sys.created_at, CHARINDEX('.',ie_sys.created_at)-1),'_',' '), 102 ) date
        FROM dbo.internal_events_sys ie_sys
        ''')
        cursor.close
        
    def drop_nulls(self, conn, db_tables_nm):
        cursor = conn.cursor()
        cursor.execute('''
         DELETE FROM ''' + db_tables_nm[3] + ''' 
         where affid is NULL or system is NULL or revenue is NULL or spent is NULL or date is NULL
        ''')
        cursor.close

### Создание объекта класса
<a id="creation_obj"></a>

In [3]:
tables = ["ad_sys", "billing_sys", "internal_events_sys", "revenue_spent_rep_table"]
files = [r'C:\\csvs_for_test_task\\ad_sys.csv', r'C:\\csvs_for_test_task\\billing_sys.csv', r'C:\\csvs_for_test_task\\internal_events.csv'] 
creator = view_creator(files, '.', 'revenue_spent_db', tables)

### Размышления и результаты
<a id="results"></a>

В первую очередь, увидев "Построить витрину" на ум пришло классическое понимание data mart'а для BI-аналитики, которую я и начал строить в SQL Server Integration Services, затем я планировал собрать табличную модель и даже сделать эти самые отчеты в Power Bi или Tableau, но затем понял, что данных недостаточно, да и задача этого не требует.
<br>
<img src="mart.png" style="width: 600px"/>

<b> Для фильтрации и аналитики необходимо: </b>
<br>
1. Выбирать конкретную систему, откуда смотреть значения (1,2,3 в столбце system)
<br>
2. Иметь возможность просмотра за различную дату и возможность интепретировать ее в разные часовые пояса (date в формате datetime) | Вычисляемыми столбцами или средствами BI-инструмента
<br>
3. Получать данные по тратам на рекламу (select spent from revenue_spent_rep_table where system = 1 (Рекламная система) )
4. Получать данные по источнику - affid

<img src="tables.png" alt="draw" style="width: 800px"/>

<b>Через pyodbc создав таблицы, импортировав их bulk insert из файлов. Итоговая таблица выглядит следующим образом:</b>
<img src="table_view.png">

Дата была обрезана до момента UTC или точки и переведена в формат datetime, строки с пустыми значениями были удалены (не знаю, подразумевалось ли это заданием)

<b> Под "создать view или представление" вряд ли подразумевалась таблица, скорее всего именно sql`ное view. Хотелось бы услышать фидбек, насколько проделанная работа соответствует ожидаемому результату и отвечает ли вообще поставленным требованиям. Заранее благодарю </b> 